## Initial Settings

In [1]:
# Load Required Libraries
import os
import sys
import platform
import pandas as pd
import numpy as np
from plydata import *

pd.set_option('display.max_columns', 100)


# Set Project Path
if platform.system() == "Linux":
    PROJECT_PATH = "/home/pooya/work/HydroTech/"
else:
    PROJECT_PATH = "c:/work/HydroTech/"


# Load Required Functions
# Function 01 - Extract 'Region', 'District' from 'Peyman' Column
def extractRD(x, para):

    if para == 'ناحیه':
        if para not in x:
            return np.nan
        elif (x.index(para) + 1) >= len(x):
            return np.nan
        else:
            return str(x[x.index(para) + 1]).zfill(2)

    if para == 'منطقه':
        if para not in x:
            if ('کمربند' in x) and ('جنوبی' in x):
                return '14'
            elif ('کمربند' in x) and ('شمالی' in x):
                return '15'
            elif ('سازمان' in x) and ('پارک‌ها' in x):
                return '16'
            else:
                return np.nan
        elif (x.index(para) + 1) >= len(x):
            return np.nan
        else:
            if str(x[x.index(para) + 1]) == 'ثامن':
                return '13'
            else:
                return str(x[x.index(para) + 1]).zfill(2)

## Load Data

In [2]:
# Read Data
raw_data = pd.read_excel(PROJECT_PATH + 'Data/Processed_Data/Merged_Data.xlsx')

raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61465 entries, 0 to 61464
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ردیف                                 61358 non-null  float64
 1   پیمان                                61465 non-null  object 
 2   تاریخ شروع پیمان                     61414 non-null  object 
 3   تاریخ پایان پیمان                    61341 non-null  object 
 4   آخرین شماره صورت وضعیت تاکنون        61341 non-null  float64
 5   نوع آیتم                             61341 non-null  object 
 6   نوع لکه                              61465 non-null  object 
 7   نام لکه                              61465 non-null  object 
 8   نوع قلم                              61465 non-null  object 
 9   زیرمجموعه هر قلم                     61465 non-null  object 
 10  مقدار در صورتجلسه تحویل و تحول       61341 non-null  float64
 11  نمایش آخرین ریزمتره (ریزمتره

# حذف ناحیه 4 کمربند جنوبی

In [3]:
raw_data = raw_data[raw_data['پیمان'] != 'ناحیه 4 کمربند جنوبی']
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61450 entries, 0 to 61464
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ردیف                                 61354 non-null  float64
 1   پیمان                                61450 non-null  object 
 2   تاریخ شروع پیمان                     61399 non-null  object 
 3   تاریخ پایان پیمان                    61341 non-null  object 
 4   آخرین شماره صورت وضعیت تاکنون        61341 non-null  float64
 5   نوع آیتم                             61341 non-null  object 
 6   نوع لکه                              61450 non-null  object 
 7   نام لکه                              61450 non-null  object 
 8   نوع قلم                              61450 non-null  object 
 9   زیرمجموعه هر قلم                     61450 non-null  object 
 10  مقدار در صورتجلسه تحویل و تحول       61341 non-null  float64
 11  نمایش آخرین ریزمتره (ریزمتره

## Error Check: Remove Duplicated Rows

In [4]:
#  Data Cleansing: Remove Duplicated Rows
# 01. Select Duplicated Rows
tmp1 = raw_data[
    raw_data.duplicated(
        subset=list(raw_data.columns)[1:],
        keep=False
    )
]


tmp1 = tmp1.sort_values(['پیمان', 'نام لکه', 'نوع قلم', 'زیرمجموعه هر قلم'])


file_name = PROJECT_PATH + "Report/v02/_00_Check_Error.xlsx"
with pd.ExcelWriter(path=file_name, engine="openpyxl", mode="w") as writer:
    tmp1.to_excel(writer, index=False, sheet_name='01_Duplicate_Rows')

    
tmp2 = raw_data[
    raw_data.duplicated(
        subset=list(raw_data.columns)[1:],
        keep='first')
]


# 02. Remove Duplicated Rows
raw_data = raw_data.drop_duplicates(
    subset=list(raw_data.columns)[1:],
    keep='first'
)


# 03. Report
print(f"Total Number of Duplicate Rows in a Data: {tmp1.shape[0]}")
print(f"Total Number of Duplicate Items in a Data: {tmp2.shape[0]}")
print("Sample of Duplicate Rows:")
display(tmp1.head(n=6))
print(f"\nData Size After Deleted Duplicate Rows: {raw_data.shape}")

del tmp1, tmp2

Total Number of Duplicate Rows in a Data: 5421
Total Number of Duplicate Items in a Data: 2720
Sample of Duplicate Rows:


,ردیف,پیمان,تاریخ شروع پیمان,تاریخ پایان پیمان,آخرین شماره صورت وضعیت تاکنون,نوع آیتم,نوع لکه,نام لکه,نوع قلم,زیرمجموعه هر قلم,مقدار در صورتجلسه تحویل و تحول,نمایش آخرین ریزمتره (ریزمتره نهایی),مساحت لکه (مترمربع),مساحت پیمان (مترمربع),میزان افزایش/کاهش ریزمتره (تعداد)
5,204.0,الف ناحیه 1 منطقه 1,1397/03/01,1400/03/01,20.0,حجمی,آیلند ها,آیلند بلوار ابوذر و لچکی ها,حفظ و نگهداری پرچین,متر مربع,1704.0,1704.0,2718.0,122292.0,0
1772,420.0,الف ناحیه 1 منطقه 1,1397/03/01,1400/03/01,20.0,حجمی,آیلند ها,آیلند بلوار ابوذر و لچکی ها,حفظ و نگهداری پرچین,متر مربع,1704.0,1704.0,2718.0,122292.0,0
6,251.0,الف ناحیه 1 منطقه 1,1397/03/01,1400/03/01,20.0,حجمی,آیلند ها,آیلند بلوار ابوذر و لچکی ها,حفظ و نگهداری چمن,متر مربع,784.0,784.0,2718.0,122292.0,0
1773,459.0,الف ناحیه 1 منطقه 1,1397/03/01,1400/03/01,20.0,حجمی,آیلند ها,آیلند بلوار ابوذر و لچکی ها,حفظ و نگهداری چمن,متر مربع,784.0,784.0,2718.0,122292.0,0
7,264.0,الف ناحیه 1 منطقه 1,1397/03/01,1400/03/01,20.0,حجمی,آیلند ها,آیلند بلوار ابوذر و لچکی ها,حفظ و نگهداری گل دائمی,متر مربع,210.0,210.0,2718.0,122292.0,0
1774,493.0,الف ناحیه 1 منطقه 1,1397/03/01,1400/03/01,20.0,حجمی,آیلند ها,آیلند بلوار ابوذر و لچکی ها,حفظ و نگهداری گل دائمی,متر مربع,210.0,210.0,2718.0,122292.0,0



Data Size After Deleted Duplicate Rows: (58730, 15)


## Extract Region, District, Peyman, Address

In [5]:
# Extract Region And District
tmp = raw_data["پیمان"].str.strip().str.split()

raw_data["Region"] = tmp.apply(extractRD, para="منطقه")

raw_data["District"] = tmp.apply(extractRD, para="ناحیه")

del tmp


# Extract Peyman
# 01. Extract Uniqe Peyman
tmp = raw_data.groupby(['Region', 'District'])['پیمان']
tmp = tmp.value_counts(dropna=False, sort=False)
tmp = pd.DataFrame(tmp)
tmp = tmp.rename(columns={'پیمان': 'Count'}).reset_index()

Peyman = []
for R in list(tmp['Region'].unique()):
    tmpR = tmp[tmp['Region'] == R]
    for D in list(tmpR['District'].unique()):
        tmpD = tmpR[tmpR['District'] == D]
        Peyman += list(range(1, len(tmpD) + 1))

tmp['Peyman'] = Peyman
tmp['Peyman'] = tmp['Peyman'].astype(str).str.zfill(2)

# 02. Add Peyman To raw_data
tmp = tmp >> select('Region', 'District', 'پیمان', 'Peyman')

raw_data = pd.merge(raw_data,
                    tmp,
                    how='left',
                    on=['Region', 'District', 'پیمان'])

del tmp, tmpR, tmpD


# Extract Address
# 01. Extract Uniqe Address
tmp = raw_data.groupby(['Region', 'District', 'Peyman'])['نام لکه']
tmp = tmp.value_counts(dropna=False, sort=False)
tmp = pd.DataFrame(tmp)
tmp = tmp.rename(columns={'نام لکه': 'Count'}).reset_index()

Address = []
for R in list(tmp['Region'].unique()):
    tmpR = tmp[tmp['Region'] == R]
    for D in list(tmpR['District'].unique()):
        tmpD = tmpR[tmpR['District'] == D]
        for P in list(tmpD['Peyman'].unique()):
            tmpP = tmpD[tmpD['Peyman'] == P]
            Address += list(range(1, len(tmpP) + 1))

tmp['Address'] = Address
tmp['Address'] = tmp['Address'].astype(str).str.zfill(3)

# 02. Add Address To raw_data
tmp = tmp >> select('Region', 'District', 'Peyman', 'نام لکه', 'Address')

raw_data = pd.merge(raw_data,
                    tmp,
                    how='left',
                    on=['Region', 'District', 'Peyman', 'نام لکه'])

del tmp, tmpR, tmpD, tmpP

## Add Address ID

In [6]:
raw_data = raw_data >> define(Address_ID = 'Region + District + Peyman + Address')

## Error Check: حذف آیتم های «حجمی» از ستون نوع آیتم

In [7]:
# 01. Select Duplicated Rows
tmp = raw_data[raw_data["نوع آیتم"] == "حجمی"]


file_name = PROJECT_PATH + "Report/v02/_00_Check_Error.xlsx"
with pd.ExcelWriter(path=file_name, engine="openpyxl", mode="a") as writer:
    tmp.to_excel(writer, index=False, sheet_name='02_Valume_Rows')


raw_data.drop(
    index=tmp.index,
    inplace=True
)

# 02. Report
print(f"Total Number of Rows == 'حجمی' in a Data: {tmp.shape[0]}")

del tmp

Total Number of Rows == 'حجمی' in a Data: 17746


## Check Region, District, Peyman, Address

In [8]:
#  Report: Check Region
display(pd.DataFrame(raw_data['Region'].value_counts(dropna=False, sort=True)).\
        reset_index().rename(columns={'index':'Region', 'Region':'Number of Rows'}).\
        sort_values(by=['Region']).reset_index(drop=True).\
        style.bar(subset=['Number of Rows'], color='red'))


#  Report: Check District
display(pd.DataFrame(raw_data['District'].value_counts(dropna=False, sort=True)).\
        reset_index().rename(columns={'index':'District', 'District':'Number of Rows'}).\
        sort_values(by=['District']).reset_index(drop=True).\
        style.bar(subset=['Number of Rows'], color='red'))


#  Report: Check Peyman
display(pd.DataFrame(raw_data['Peyman'].value_counts(dropna=False, sort=True)).\
        reset_index().rename(columns={'index':'Peyman', 'Peyman':'Number of Rows'}).\
        sort_values(by=['Peyman']).reset_index(drop=True).\
        style.bar(subset=['Number of Rows'], color='red'))


#  Report: Check Address
display(pd.DataFrame(raw_data['Address'].value_counts(dropna=False, sort=True)).\
        reset_index().rename(columns={'index':'Address', 'Address':'Number of Rows'}).\
        sort_values(by=['Address']).reset_index(drop=True).\
        style.bar(subset=['Number of Rows'], color='red'))

,Region,Number of Rows
0,01,4855
1,02,4911
2,03,1991
3,04,1166
4,05,2217
5,06,2203
6,07,3745
7,08,2271
8,09,4836
9,10,5742


,District,Number of Rows
0,01,14988
1,02,16424
2,03,7685
3,04,1839
4,10.2,8
5,11.2,8
6,12.2,4
7,7.1,28


,Peyman,Number of Rows
0,01,27016
1,02,10075
2,03,3136
3,04,637
4,05,120


,Address,Number of Rows
0,001,1282
1,002,1155
2,003,1182
3,004,1077
4,005,1069
5,006,1058
6,007,1097
7,008,1076
8,009,997
9,010,968


## Spots Class

In [9]:
# 01. Define Spots Class
Spots_Class = {
    'میادین': '01',
    'لچکی ها': '02',
    'آیلند های بزرگراه': '03',
    'آیلند ها': '04',
    'حاشیه های بزرگراه': '05',
    'حاشیه معابر': '06',
    'بوستان خطی': '07',
    'پارک های زیر 6 هکتار': '08',
    'پارک های بین 6 تا 10 هکتار': '09',
    'پارک های بالای 10 هکتار': '10',
    'جنگل کاری داخل محدوده': '11',
    'کمربندی': '12',
    'کمربند سبز حفاظتی': '13',
}


# 02. Extract All Spots Class From Data
tmp = raw_data['نوع لکه'].value_counts(dropna=False, sort=True).reset_index()
tmp = tmp.rename(columns={'index' : 'Name of Spot',
                          'نوع لکه' : 'Number of Rows'})

tmp['Spot'] = list(map(Spots_Class.get, tmp['Name of Spot']))
tmp = tmp[['Spot', 'Name of Spot', 'Number of Rows']]


# 03. Report
display(tmp.sort_values(by=['Spot']).reset_index(drop=True).\
        style.bar(subset=['Number of Rows'], color='red'))


# 04. Add Spots Class to raw_data
tmp = tmp[['Spot', 'Name of Spot']]
raw_data = pd.merge(raw_data,
                    tmp,
                    how='left',
                    left_on=['نوع لکه'],
                    right_on=['Name of Spot']).drop('Name of Spot', axis=1)


del tmp

,Spot,Name of Spot,Number of Rows
0,01,میادین,2481
1,02,لچکی ها,5020
2,03,آیلند های بزرگراه,614
3,04,آیلند ها,5221
4,05,حاشیه های بزرگراه,918
5,06,حاشیه معابر,10880
6,08,پارک های زیر 6 هکتار,13276
7,09,پارک های بین 6 تا 10 هکتار,168
8,10,پارک های بالای 10 هکتار,883
9,11,جنگل کاری داخل محدوده,1437


## Irrigation Method Class

In [10]:
# 01. Define Irrigation Method Class
Irrigation_Method_Class = {
    'آبیاری ثقلی': '01',
    'آبیاری تانکری': '02',
    'آبیاری شلنگی': '03',
    'آبیاری تحت فشار': '04'
}


# 02. Extract All Irrigation Method Class From Data
tmp = raw_data['زیرمجموعه هر قلم'].value_counts(dropna=False, sort=True).reset_index()
tmp = tmp.rename(columns={'index': 'Name of Item',
                          'زیرمجموعه هر قلم': 'Number of Rows'})

tmp['Irrigation'] = list(map(Irrigation_Method_Class.get, tmp['Name of Item']))
tmp = tmp[['Irrigation', 'Name of Item', 'Number of Rows']]


# 03. Report
display(tmp.sort_values(by=['Irrigation']).reset_index(drop=True).\
        style.bar(subset=['Number of Rows'], color='red'))


# 04. Add Irrigation Method Class to raw_data
tmp = tmp[['Name of Item', 'Irrigation']]
raw_data = pd.merge(raw_data,
                    tmp,
                    how='left',
                    left_on=['زیرمجموعه هر قلم'],
                    right_on=['Name of Item']).drop('Name of Item', axis=1)

# 05. Add Irrigation Area to raw_data
tmp = list(Irrigation_Method_Class.values())

def GetIrrgAreaValue(row):
    if row['Irrigation'] in tmp:
        return row['نمایش آخرین ریزمتره (ریزمتره نهایی)']
    else:
        return np.nan

raw_data['Irrigation_Area'] = raw_data.apply(GetIrrgAreaValue, axis=1)


raw_data.fillna(value=np.nan, inplace=True)


del tmp

,Irrigation,Name of Item,Number of Rows
0,01,آبیاری ثقلی,194
1,02,آبیاری تانکری,1227
2,03,آبیاری شلنگی,5439
3,04,آبیاری تحت فشار,539
4,None,متر مربع,6761
5,None,شستشو,5239
6,None,سمپاشی,5162
7,None,محلول پاشی,3670
8,None,اصله,2215
9,None,حذف تنه جوش و پاجوش,2091


## Species Plant Class

In [11]:
# 01. Define Species Plant Class
Species_Plant_Class = {
    'چمن': '01',
    'گل دائم باغچه های معمولی': '02',
    'گل دائم فلاورباکسهای سطوح شیب دار': '04',
    'گل فصل باغچه های معمولی': '05',
    'پرچین': '06',
    'درخت و درختچه': '10',
    'درختان جنگلی': '12',
    'گل فصل فلاورباکس های سطوح شیب دار': '14'
}


# 02. Extract All Species Plant Class From Data
tmp = raw_data['نوع قلم'].value_counts(dropna=False, sort=True).reset_index()
tmp = tmp.rename(columns={'index': 'Name of Species',
                          'نوع قلم' : 'Number of Rows'})

tmp['Species'] = list(map(Species_Plant_Class.get, tmp['Name of Species']))
tmp = tmp[['Species', 'Name of Species', 'Number of Rows']]


# 03. Report
display(tmp.sort_values(by=['Species']).reset_index(drop=True).\
        style.bar(subset=['Number of Rows'], color='red'))

# 04. Add Species Plant Class to raw_data
tmp = tmp[['Species', 'Name of Species']]
raw_data = pd.merge(raw_data,
                    tmp,
                    how='left',
                    left_on=['نوع قلم'],
                    right_on=['Name of Species']).drop('Name of Species', axis=1)


del tmp

,Species,Name of Species,Number of Rows
0,01,چمن,2353
1,02,گل دائم باغچه های معمولی,5431
2,04,گل دائم فلاورباکسهای سطوح شیب دار,57
3,05,گل فصل باغچه های معمولی,1997
4,06,پرچین,3397
5,10,درخت و درختچه,27696
6,12,درختان جنگلی,45
7,14,گل فصل فلاورباکس های سطوح شیب دار,8


## Generate ID

In [12]:
# 01. Check Number of NaN
display(pd.DataFrame(raw_data[['Region', 'District', 'Peyman', 'Address',
                               'Spot', 'Irrigation', 'Species']].isnull().sum()).\
        rename(columns={0:'Number of NaN'}))


# 02. Generate ID
tmp = 'Region + District + Peyman + Address + "-" + Spot + Irrigation + Species'
raw_data = raw_data >> define(ID=tmp)


# 03. ID Check
tmp = raw_data.astype(str).groupby(['ID']).size()
tmp = pd.DataFrame(tmp).rename(columns={0: 'Count'})
tmp = tmp >> query("Count >= 2")
print(f"\nNumber of Duplicate ID: {tmp.shape[0]}")
tmp = list(tmp.index[[0, 1, 2]])
print("\nSample of Duplicate ID:")
display(raw_data >> query('ID in @tmp'))


# 04. Save Duplicate ID
tmp = raw_data.dropna(subset=['ID']).duplicated(subset="ID", keep=False)
tmp = raw_data.dropna(subset=['ID'])[tmp.values]

file_name = PROJECT_PATH + "Report/v02/_00_Check_Error.xlsx"
with pd.ExcelWriter(path=file_name, engine="openpyxl", mode="a") as writer:
    tmp.to_excel(writer, index=False, sheet_name='03_Duplicate_ID')
    
# 05. Remove
tmp = raw_data.dropna(subset=['ID']).duplicated(subset="ID", keep='last')
tmp = tmp[tmp.values == True].index
raw_data.drop(axis=0, index=tmp, inplace=True)

,Number of NaN
Region,0
District,0
Peyman,0
Address,0
Spot,0
Irrigation,33585
Species,0



Number of Duplicate ID: 90

Sample of Duplicate ID:


,ردیف,پیمان,تاریخ شروع پیمان,تاریخ پایان پیمان,آخرین شماره صورت وضعیت تاکنون,نوع آیتم,نوع لکه,نام لکه,نوع قلم,زیرمجموعه هر قلم,مقدار در صورتجلسه تحویل و تحول,نمایش آخرین ریزمتره (ریزمتره نهایی),مساحت لکه (مترمربع),مساحت پیمان (مترمربع),میزان افزایش/کاهش ریزمتره (تعداد),Region,District,Peyman,Address,Address_ID,Spot,Irrigation,Irrigation_Area,Species,ID
8158,166.0,ناحیه 3 منطقه 2,1396/09/01,1399/09/02,26.0,ریزآیتمی,جنگل کاری داخل محدوده,جنگل 80 متری معراج,درخت و درختچه,آبیاری ثقلی,1371.0,1680.0,28500.0,134017.0,309,02,03,01,016,020301016,11,01,1680.0,10,020301016-110110
8159,230.0,ناحیه 3 منطقه 2,1396/09/01,1399/09/02,26.0,ریزآیتمی,جنگل کاری داخل محدوده,جنگل 80 متری معراج,درخت و درختچه,آبیاری ثقلی,28500.0,28500.0,28500.0,134017.0,0,02,03,01,016,020301016,11,01,28500.0,10,020301016-110110
9485,394.0,ب ناحیه 4 منطقه 2 - 9803,1398/03/01,1401/03/01,8.0,ریزآیتمی,پارک های زیر 6 هکتار,پارک خطی کارگر,درخت و درختچه,آبیاری ثقلی,194.0,194.0,14852.0,271334.0,0,02,04,02,039,020402039,08,01,194.0,10,020402039-080110
9486,553.0,ب ناحیه 4 منطقه 2 - 9803,1398/03/01,1401/03/01,8.0,ریزآیتمی,پارک های زیر 6 هکتار,پارک خطی کارگر,درخت و درختچه,آبیاری ثقلی,1200.0,1200.0,14852.0,271334.0,0,02,04,02,039,020402039,08,01,1200.0,10,020402039-080110
10494,289.0,ناحیه 2 منطقه 3 - 9810,1398/10/01,1401/10/01,1.0,ریزآیتمی,پارک های زیر 6 هکتار,بوستان سیس آباد,درخت و درختچه,آبیاری ثقلی,1000.0,1000.0,27339.0,257206.0,0,03,02,01,015,030201015,08,01,1000.0,10,030201015-080110
10495,309.0,ناحیه 2 منطقه 3 - 9810,1398/10/01,1401/10/01,1.0,ریزآیتمی,پارک های زیر 6 هکتار,بوستان سیس آباد,درخت و درختچه,آبیاری ثقلی,234.0,234.0,27339.0,257206.0,0,03,02,01,015,030201015,08,01,234.0,10,030201015-080110


## Extract Volume Data

In [13]:
# 00. Load Volume Data
Volume_Data = pd.read_excel(PROJECT_PATH + "Report/v02/_00_Check_Error.xlsx",
                            sheet_name='02_Valume_Rows',
                            dtype={'Region': str,
                                   'District': str,
                                   'Peyman' : str,
                                   'Address' : str,
                                   'Address_ID' : str})



# 01. Extract Maintenance Rows
Volume_Data = Volume_Data[Volume_Data["نوع قلم"].str.contains("حفظ و نگهداری", na=False)]


# 02. Define Maintenance Class
Maintenance_Class = {
    'حفظ و نگهداری چمن': '01',
    'حفظ و نگهداری گل دائمی': '02',
    'حفظ و نگهداری سطوح شیبدار گل دائمی': '04',
    'حفظ و نگهداری گل فصلی': '05',
    'حفظ و نگهداری پرچین': '06',
    'حفظ و نگهداری درخت و درختچه': '10',
    'حفظ و نگهداری سطوح شیبدار گل فصلی': '14'
}


# 02. Extract All Species Plant Class From Data
tmp = Volume_Data['نوع قلم'].value_counts(dropna=False, sort=True).reset_index()
tmp = tmp.rename(columns={'index': 'Name of Species',
                          'نوع قلم' : 'Number of Rows'})

tmp['Species'] = list(map(Maintenance_Class.get, tmp['Name of Species']))
tmp = tmp[['Species', 'Name of Species', 'Number of Rows']]


# 03. Report
display(tmp.sort_values(by=['Species']).reset_index(drop=True).\
        style.bar(subset=['Number of Rows'], color='red'))

# 04. Add Maintenance Class to Volume_Data
tmp = tmp[['Species', 'Name of Species']]
Volume_Data = pd.merge(Volume_Data,
                       tmp,
                       how='left',
                       left_on=['نوع قلم'],
                       right_on=['Name of Species']).drop('Name of Species', axis=1)

# 05. Add Maintenance_ID
Volume_Data = Volume_Data >> define(Maintenance_ID = 'Region + District + Peyman + Address + Species')


del tmp

,Species,Name of Species,Number of Rows
0,01,حفظ و نگهداری چمن,1094
1,02,حفظ و نگهداری گل دائمی,1414
2,04,حفظ و نگهداری سطوح شیبدار گل دائمی,16
3,05,حفظ و نگهداری گل فصلی,940
4,06,حفظ و نگهداری پرچین,1572
5,10,حفظ و نگهداری درخت و درختچه,1480
6,14,حفظ و نگهداری سطوح شیبدار گل فصلی,4


## Meter Error

In [14]:
# 01. Find Meter Error
tmp = raw_data[(raw_data['نوع قلم'] == 'درخت و درختچه') & (raw_data['زیرمجموعه هر قلم'] == 'متر')] 


# 02. Report
display(tmp.head(4))


# 03. Save Meter Error
file_name = PROJECT_PATH + "Report/v02/_00_Check_Error.xlsx"
with pd.ExcelWriter(path=file_name, engine="openpyxl", mode="a") as writer:
    tmp.to_excel(writer, index=False, sheet_name='04_Meter_Error')


# 04. Modified Meter Error
raw_data.loc[list(tmp.index), "زیرمجموعه هر قلم"] = "متر مربع"


del tmp, file_name

,ردیف,پیمان,تاریخ شروع پیمان,تاریخ پایان پیمان,آخرین شماره صورت وضعیت تاکنون,نوع آیتم,نوع لکه,نام لکه,نوع قلم,زیرمجموعه هر قلم,مقدار در صورتجلسه تحویل و تحول,نمایش آخرین ریزمتره (ریزمتره نهایی),مساحت لکه (مترمربع),مساحت پیمان (مترمربع),میزان افزایش/کاهش ریزمتره (تعداد),Region,District,Peyman,Address,Address_ID,Spot,Irrigation,Irrigation_Area,Species,ID
8167,226.0,ناحیه 3 منطقه 2,1396/09/01,1399/09/02,26.0,ریزآیتمی,جنگل کاری داخل محدوده,جنگل 80 متری معراج,درخت و درختچه,متر,5900.0,5900.0,28500.0,134017.0,0,02,03,01,016,020301016,11,NaN,NaN,10,NaN
10501,310.0,ناحیه 2 منطقه 3 - 9810,1398/10/01,1401/10/01,1.0,ریزآیتمی,پارک های زیر 6 هکتار,بوستان سیس آباد,درخت و درختچه,متر,1360.0,1360.0,27339.0,257206.0,0,03,02,01,015,030201015,08,NaN,NaN,10,NaN
10686,311.0,ناحیه 2 منطقه 3 - 9810,1398/10/01,1401/10/01,1.0,ریزآیتمی,حاشیه های بزرگراه,حاشیه بزرگراه صدمتری,درخت و درختچه,متر,570.0,570.0,24851.0,257206.0,0,03,02,01,022,030201022,05,NaN,NaN,10,NaN
10856,388.0,ناحیه 3 منطقه 3 - 9808,1398/08/01,1401/08/01,3.0,ریزآیتمی,آیلند های بزرگراه,وسط صدمتری و سوزنی برگان,درخت و درختچه,متر,296.0,296.0,10865.0,392697.0,0,03,03,01,015,030301015,03,NaN,NaN,10,NaN


## Add Some Element To Database

In [15]:
# 01. Define Extera Class
Extera_Class = {
    'متر مربع' : '99',
    'اصله' : '00'
}


# 02. Extract All Extera Class In Data
tmp = raw_data['زیرمجموعه هر قلم'].value_counts(dropna=False, sort=True)
tmp = tmp.reset_index().rename(columns={'index': 'Subset Item',
                                        'زیرمجموعه هر قلم': 'Number of Rows'})
tmp['Extera'] = list(map(Extera_Class.get, tmp['Subset Item']))
tmp = tmp[['Extera', 'Subset Item', 'Number of Rows']]


# 03. Report
display(tmp.sort_values(by=['Extera']).reset_index(drop=True).\
        style.bar(subset=['Number of Rows'], color='red'))


# 04. Add Extera Class to data
tmp = tmp[['Extera', 'Subset Item']]
raw_data = pd.merge(raw_data,
                    tmp,
                    how='left',
                    left_on=['زیرمجموعه هر قلم'],
                    right_on=['Subset Item']).drop('Subset Item', axis=1)
raw_data.fillna(value=np.nan, inplace=True)


# 05. Add Extera Value to data
tmp = list(Extera_Class.values())

def GetExteraValue(row):
    if row['Extera'] in tmp:
        return row['نمایش آخرین ریزمتره (ریزمتره نهایی)']
    else:
        return np.nan

raw_data['Extera_Value'] = raw_data.apply(GetExteraValue, axis=1)


del tmp

,Extera,Subset Item,Number of Rows
0,00,اصله,2215
1,99,متر مربع,6857
2,None,آبیاری شلنگی,5439
3,None,شستشو,5239
4,None,سمپاشی,5162
5,None,محلول پاشی,3670
6,None,حذف تنه جوش و پاجوش,2091
7,None,هرس زمستانه,2089
8,None,هرس سبز,2071
9,None,هرس فرم,1753


## Modify ID

In [16]:
s = 'Region + District + Peyman + Address + "-" + Spot + "00" + Species'
raw_data = raw_data >> define(ID=if_else('Extera == "00"', s, 'ID'))

s = 'Region + District + Peyman + Address + "-" + Spot + "99" + Species'
raw_data = raw_data >> define(ID=if_else('Extera == "99"', s, 'ID'))

raw_data['ID'].replace(to_replace='nan', value=np.NaN, inplace=True)

## Remove Duplicate ID

In [17]:
# Save Duplicate ID
tmp = raw_data.dropna(subset=['ID']).duplicated(subset="ID", keep=False)
tmp = raw_data.dropna(subset=['ID'])[tmp.values]

file_name = PROJECT_PATH + "Report/v02/_00_Check_Error.xlsx"
with pd.ExcelWriter(path=file_name, engine="openpyxl", mode="a") as writer:
    tmp.to_excel(writer, index=False, sheet_name='05_Duplicate_Modify_ID')

In [19]:
# Remove Duplicate ID
# TODO: Find Beter Way
tmp = raw_data.dropna(subset=['ID']).duplicated(subset="ID", keep='last')
tmp = tmp[tmp.values == True].index
raw_data.drop(axis=0, index=tmp, inplace=True)

## Change Columns Name of raw_data

In [20]:
# 01. Select Data
data = raw_data >> select('ID', 'Region', 'District', 'Peyman', 'Address', 'Spot', 'Irrigation', 'Species',
                          'Extera',
                          'نمایش آخرین ریزمتره (ریزمتره نهایی)',
                          'Irrigation_Area', 'Extera_Value',
                          'مساحت لکه (مترمربع)', 
                          'مساحت پیمان (مترمربع)',
                          'نوع لکه',
                          'نام لکه',
                          'نوع قلم',
                          'زیرمجموعه هر قلم',
                          'پیمان',
                          'میزان افزایش/کاهش ریزمتره (تعداد)')


# 02. Rename Columns
data.rename(columns={
    'نمایش آخرین ریزمتره (ریزمتره نهایی)': 'Item_Area',
    'مساحت لکه (مترمربع)': 'Address_Area',
    'مساحت پیمان (مترمربع)': 'Peyman_Area',
    'نوع لکه': 'Address_Type',
    'نام لکه': 'Address_Name',
    'نوع قلم': 'Item_Type',
    'زیرمجموعه هر قلم': 'Subset_Item',
    'پیمان': 'Peyman_Name',
    'میزان افزایش/کاهش ریزمتره (تعداد)' : 'DecreaseـIncrease_Rizmetre'
}, inplace=True)


# 03. Save Data
file_name = PROJECT_PATH + "Report/v02/Data.xlsx"
with pd.ExcelWriter(path=file_name, engine="openpyxl", mode="w") as writer:
    data.sort_values(by=['Region', 'District', 'Peyman', 'Address']).to_excel(writer, index=False, sheet_name='data')


print(f"Size of Modified Data: {data.shape}")


data

Size of Modified Data: (40800, 20)


,ID,Region,District,Peyman,Address,Spot,Irrigation,Species,Extera,Item_Area,Irrigation_Area,Extera_Value,Address_Area,Peyman_Area,Address_Type,Address_Name,Item_Type,Subset_Item,Peyman_Name,DecreaseـIncrease_Rizmetre
0,010101001-040206,01,01,01,001,04,02,06,NaN,1405.0,1405.0,NaN,2718.0,122292.0,آیلند ها,آیلند بلوار ابوذر و لچکی ها,پرچین,آبیاری تانکری,الف ناحیه 1 منطقه 1,0
1,010101001-040306,01,01,01,001,04,03,06,NaN,299.0,299.0,NaN,2718.0,122292.0,آیلند ها,آیلند بلوار ابوذر و لچکی ها,پرچین,آبیاری شلنگی,الف ناحیه 1 منطقه 1,0
2,010101001-049906,01,01,01,001,04,NaN,06,99,1704.0,NaN,1704.0,2718.0,122292.0,آیلند ها,آیلند بلوار ابوذر و لچکی ها,پرچین,متر مربع,الف ناحیه 1 منطقه 1,0
3,010101001-040301,01,01,01,001,04,03,01,NaN,784.0,784.0,NaN,2718.0,122292.0,آیلند ها,آیلند بلوار ابوذر و لچکی ها,چمن,آبیاری شلنگی,الف ناحیه 1 منطقه 1,0
4,010101001-049901,01,01,01,001,04,NaN,01,99,784.0,NaN,784.0,2718.0,122292.0,آیلند ها,آیلند بلوار ابوذر و لچکی ها,چمن,متر مربع,الف ناحیه 1 منطقه 1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40890,167.101001-090306,16,7.1,01,001,09,03,06,NaN,5612.0,5612.0,NaN,82094.0,256862.0,پارک های بین 6 تا 10 هکتار,پارک ریحانه,پرچین,آبیاری شلنگی,ناحیه 7.1 سازمان پارک‌ها,NaN
40891,167.101001-099906,16,7.1,01,001,09,NaN,06,99,5612.0,NaN,5612.0,82094.0,256862.0,پارک های بین 6 تا 10 هکتار,پارک ریحانه,پرچین,متر مربع,ناحیه 7.1 سازمان پارک‌ها,NaN
40892,167.101001-090310,16,7.1,01,001,09,03,10,NaN,3936.0,3936.0,NaN,82094.0,256862.0,پارک های بین 6 تا 10 هکتار,پارک ریحانه,درخت و درختچه,آبیاری شلنگی,ناحیه 7.1 سازمان پارک‌ها,NaN
40893,167.101001-090210,16,7.1,01,001,09,02,10,NaN,54487.0,54487.0,NaN,82094.0,256862.0,پارک های بین 6 تا 10 هکتار,پارک ریحانه,درخت و درختچه,آبیاری تانکری,ناحیه 7.1 سازمان پارک‌ها,NaN


## Error Check سایر گل های دائمی And گل رز 

In [21]:
# 01. Roze and Other Flowers
tmp1 = data[(data['Subset_Item'] == "گل رز") | (data['Subset_Item'] == "سایر گل های دائمی")]
tmp1 = pd.DataFrame(tmp1.groupby(['Region', 'District', 'Peyman', 'Address', 'Species'])['Item_Area'].sum())
tmp1 = tmp1.reset_index().rename(columns={'Item_Area' : 'Sum_Roze_OtherFlowers'})


# 02. Evergreen Flowers
tmp2 = data[(data['Extera'] == "99") & ((data['Species'] == "02") | (data['Species'] == "04"))]
tmp2 = pd.DataFrame(tmp2.groupby(['Region', 'District', 'Peyman', 'Address', 'Species'])['Item_Area'].sum())
tmp2 = tmp2.reset_index().rename(columns={'Item_Area' : 'Sum_EvergreenFlowers'})


# 03. Merged
tmp = tmp1.merge(tmp2, how='outer', on=['Region', 'District', 'Peyman', 'Address', 'Species'])
tmp['Different'] = tmp['Sum_EvergreenFlowers'] - tmp['Sum_Roze_OtherFlowers']


# 04. Save Error
file_name = PROJECT_PATH + "Report/v02/_00_Check_Error.xlsx"
with pd.ExcelWriter(path=file_name, engine="openpyxl", mode="a") as writer:
    tmp.to_excel(writer, index=False, sheet_name='06_Roze_OtherFlowers_Error')

## Error Check Asle

In [22]:
# استخراج تعداد اصله - متر مربع اصله - آبیاری اصله
tmp = data >> call('.dropna', subset=['ID'])
tmp = tmp >> define(Category = if_else('pd.isna(Irrigation)', 'Extera', '55'))
tmp = tmp[(tmp['Species'] == "10") | (tmp['Species'] == "12")]
tmp = tmp.pivot_table(values=["Item_Area"],
                index=['Region', 'District', 'Peyman', 'Address'],
                columns=['Category'],
                aggfunc=['sum']).reset_index().to_excel(PROJECT_PATH + "Report/v02/Asle.xlsx")
tmp = pd.read_excel(PROJECT_PATH + "Report/v02/Asle.xlsx", dtype={'Region': str,
                                                              'District': str,
                                                             'Peyman' : str,
                                                             'Address' : str})
tmp = tmp.rename(columns={'sum' : 'Asle',
                          'Unnamed: 6' : 'Irrigation',
                         'Unnamed: 7' : 'SquareMetre'}).drop('Unnamed: 0', axis=1)
tmp = tmp.drop(tmp.index[0], axis=0)
tmp = tmp.drop(tmp.index[0], axis=0)
tmp = tmp.drop(tmp.index[0], axis=0)

addArea = data >> call('.dropna', subset=['ID'])
addArea = addArea >> define(Category = if_else('pd.isna(Irrigation)', 'Extera', '55'))
addArea = addArea[(addArea['Species'] == "10") | (addArea['Species'] == "12")]
addArea = addArea.groupby(['Region', 'District', 'Peyman', 'Address'])
addArea = addArea.agg({'Address_Area' : ['mean']}).reset_index()
addArea.columns = ['Region', 'District', 'Peyman', 'Address', 'addArea']

tmp = tmp.merge(addArea, how='outer', on=['Region', 'District', 'Peyman', 'Address'])


# استخراج حفظ و نگهداری درخت و درختچه
Volume_Data = pd.read_excel(PROJECT_PATH + "Report/v02/_00_Check_Error.xlsx",
                            sheet_name='02_Valume_Rows',
                            dtype={'Region': str,
                                   'District': str,
                                   'Peyman' : str,
                                   'Address' : str})

Volume_Data = Volume_Data[Volume_Data['نوع قلم'] == 'حفظ و نگهداری درخت و درختچه']
Volume_Data = Volume_Data.groupby(['Region', 'District', 'Peyman', 'Address'])
Volume_Data = Volume_Data['نمایش آخرین ریزمتره (ریزمتره نهایی)'].sum().reset_index()
Volume_Data = Volume_Data.rename(columns={'نمایش آخرین ریزمتره (ریزمتره نهایی)':'Maintenance'})


# ترکیب کردن اصله و حفظ و نگهداری
asle_data_check = tmp.merge(Volume_Data, how='outer', on=['Region', 'District', 'Peyman', 'Address'])


# استخراج سطوح غیر سبز
Volume_Data = pd.read_excel(PROJECT_PATH + "Report/v02/_00_Check_Error.xlsx",
                            sheet_name='02_Valume_Rows',
                            dtype={'Region': str,
                                   'District': str,
                                   'Peyman' : str,
                                   'Address' : str})
x = ['آبنما نظافت', 'فضاهاي مسقف', 'معابر', 'محوطه بازي كودكان']
Maaber = Volume_Data[Volume_Data['نوع قلم'].isin(x)]
Maaber = Maaber[Maaber['زیرمجموعه هر قلم'].isin(['متر مربع'])]
Maaber = Maaber.groupby(['Region', 'District', 'Peyman', 'Address'])
Maaber = Maaber['نمایش آخرین ریزمتره (ریزمتره نهایی)'].sum().reset_index()
Maaber = Maaber.rename(columns={'نمایش آخرین ریزمتره (ریزمتره نهایی)':'Maaber'})


# اضافه کردن سطوح غیر سبز به اصله و حفظ و نگهداری
Result_Area = asle_data_check.merge(Maaber, how='outer', on=['Region', 'District', 'Peyman', 'Address'])


# استخراج مساحت آبیاری سایر گونه های به غیر از درخت و درختچه
tmp = data >> call('.dropna', subset=['Irrigation'])
tmp = tmp[(tmp['Species'] != "10") & (tmp['Species'] != "12")]
tmp = tmp.groupby(['Region', 'District', 'Peyman', 'Address'])
tmp = tmp.agg({'Irrigation_Area' : ['sum'],
               'Address_Area' : ['mean']})
Irrigation_Area_No_Asle = tmp.reset_index()
Irrigation_Area_No_Asle.columns = ['Region', 'District', 'Peyman', 'Address', 'Irrigation_Area_No_Asle', 'Address_Area']


# اضافه کردن به داده ها
Result_Area = Result_Area.merge(Irrigation_Area_No_Asle, how='outer', on=['Region', 'District', 'Peyman', 'Address'])

# مساحت سایر گونه ها
Other_Species_m2 = data >> query('Extera == "99"') >> query('Species != "10" & Species != "12"')
Other_Species_m2 = Other_Species_m2.groupby(['Region', 'District', 'Peyman', 'Address'])['Item_Area']
Other_Species_m2 = pd.DataFrame(Other_Species_m2.sum()).reset_index().rename(columns={'Item_Area' : 'Other_Species'})

Result_Area = Result_Area.merge(Other_Species_m2, how='outer', on=['Region', 'District', 'Peyman', 'Address'])


def aa(x, y):
    if pd.isna(x) and pd.isna(y):
        return np.nan
    elif pd.notna(x) and pd.notna(y):
        return x
    elif pd.isna(x):
        return y
    elif pd.isna(y):
        return x

Result_Area['Address_Area'] = Result_Area.apply(lambda row : aa(row['addArea'], row['Address_Area']), axis = 1)
Result_Area = Result_Area.drop('addArea', axis=1)


Result_Area.sort_values(by=['Region', 'District', 'Peyman', 'Address']).to_excel(PROJECT_PATH + "Report/v02/Result_Area.xlsx", index=False)


## چک کردن مساحت ها

In [23]:
Result_Area['result'] = Result_Area['Address_Area'].replace(to_replace=np.NaN, value=0) -\
(Result_Area['SquareMetre'].replace(to_replace=np.NaN, value=0) +\
 Result_Area['Maaber'].replace(to_replace=np.NaN, value=0) +\
 Result_Area['Other_Species'].replace(to_replace=np.NaN, value=0))

Result_Area['result'].replace(to_replace=0, value=np.NaN, inplace=True)

tmp1 = Result_Area.dropna(subset=['result']) >> define(ADD_ID = 'Region + District + Peyman + Address')

file_name = PROJECT_PATH + "Report/v02/_00_Check_Error.xlsx"
with pd.ExcelWriter(path=file_name, engine="openpyxl", mode="a") as writer:
    tmp1.sort_values(by=['Region', 'District', 'Peyman', 'Address']).to_excel(writer, index=False, sheet_name='07_Area_Error_Address')
    
tmp2 = data >> define(ADD_ID = 'Region + District + Peyman + Address')
tmp2 = tmp2.loc[tmp2['ADD_ID'].isin(tmp1['ADD_ID'])]

file_name = PROJECT_PATH + "Report/v02/_00_Check_Error.xlsx"
with pd.ExcelWriter(path=file_name, engine="openpyxl", mode="a") as writer:
    tmp2.sort_values(by=['Region', 'District', 'Peyman', 'Address']).to_excel(writer, index=False, sheet_name='08_Area_Error_Data')

In [24]:
#  Report: Check Region, District and Peyman
tmpG = data.astype(str).groupby(
    ['Region', 'District', 'Peyman', 'Peyman_Name'])
tmp = pd.DataFrame(tmpG.size())
tmp = tmp.reset_index()
tmp = tmp.rename(columns={'Region': 'منطقه',
                          'District': 'ناحیه',
                          'Peyman': 'پیمان',
                          'Peyman_Name': 'نام پیمان',
                          0: 'تعداد ردیف'})

file_name = PROJECT_PATH + "Report/v02/Region_District_Peyman.xlsx"
tmp.to_excel(file_name, index=False)

print(tmp)
del tmpG, tmp, file_name

#  Report: Check Region, District, Peyman, Address
tmpG = data.astype(str).groupby(
    ['Region', 'District', 'Peyman', 'Address', 'Address_Name']
)
tmp = pd.DataFrame(tmpG.size())
tmp = tmp.reset_index()
tmp = tmp.rename(columns={'Region': 'منطقه',
                          'District': 'ناحیه',
                          'Peyman': 'پیمان',
                          'Address': 'لکه',
                          'Address_Name': 'نام لکه',
                          0: 'تعداد ردیف'})

file_name = PROJECT_PATH + "Report/v02/Region_District_Peyman_Address.xlsx"
tmp.to_excel(file_name, index=False)

print(tmp)
del tmpG, tmp, file_name


#  Report: Species Plant Class In Data
tmp = data.groupby(['Item_Type', 'Subset_Item']).size()
tmp = tmp.reset_index()
tmp = tmp.rename(columns={'Item_Type': 'نوع قلم',
                          'Subset_Item': 'زیرمجموعه هر قلم',
                          0: 'تعداد ردیف'})

file_name = PROJECT_PATH + '/Report/v02/Ghalam_SubGhalam.xlsx'
tmp.to_excel(file_name, index=False)

del tmp, file_name

tmp = data.groupby(['Subset_Item', 'Item_Type']).size()
tmp = tmp.reset_index()
tmp = tmp.rename(columns={'Item_Type': 'نوع قلم',
                          'Subset_Item': 'زیرمجموعه هر قلم',
                          0: 'تعداد ردیف'})

file_name = PROJECT_PATH + '/Report/v02/SubGhalam_Ghalam.xlsx'
tmp.to_excel(file_name, index=False)

del tmp, file_name


#  Report: Irrigation - Species - RDPA
tmpG = data.groupby(
    ['Region', 'District', 'Peyman', 'Address', 'Irrigation', 'Species']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum',
    'Address_Area': 'mean',
    'Peyman_Area': 'mean'
})

file_name = PROJECT_PATH + "Report/v02/Report_RDPAIS_Irrigated_Address_Peyman_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report: Irrigation - Species - RDP
tmpG = data.groupby(
    ['Region', 'District', 'Peyman', 'Irrigation', 'Species']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum'
})

file_name = PROJECT_PATH + "Report/v02/Report_RDPIS_Irrigated_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report: Species - Irrigation - RDP
tmpG = data.groupby(
    ['Region', 'District', 'Peyman', 'Species', 'Irrigation']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum'
})

file_name = PROJECT_PATH + "Report/v02/Report_RDPSI_Irrigated_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report: Irrigation - Species - RD
tmpG = data.groupby(
    ['Region', 'District', 'Irrigation', 'Species']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum'
})

file_name = PROJECT_PATH + "Report/v02/Report_RDIS_Irrigated_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report:  Species - Irrigation - RD
tmpG = data.groupby(
    ['Region', 'District', 'Species', 'Irrigation']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum'
})

file_name = PROJECT_PATH + "Report/v02/Report_RDSI_Irrigated_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report: Irrigation - Species - R
tmpG = data.groupby(
    ['Region', 'Irrigation', 'Species']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum'
})

file_name = PROJECT_PATH + "Report/v02/Report_RIS_Irrigated_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report: Irrigation - Species - R
tmpG = data.groupby(
    ['Region', 'Species', 'Irrigation']
)

tmp = tmpG.agg({
    'Irrigation_Area': 'sum'
})

file_name = PROJECT_PATH + "Report/v02/Report_RSI_Irrigated_Area.xlsx"

tmp.to_excel(file_name, index=True)

del tmpG, tmp, file_name


#  Report: Irrigation - Species
tmp = data.pivot_table(values="Irrigation_Area",
                       index=['Species'],
                       columns=['Irrigation'],
                       aggfunc=['count', 'sum'],
                       margins=True,
                       margins_name="Total")

file_name = PROJECT_PATH + "Report/v02/PivotTable_Species_Irrigation_Count_Area.xlsx"
tmp.to_excel(file_name)


#  Report: Irrigation - Species - Region
tmp = data.pivot_table(values="Irrigation_Area",
                       index=['Region', 'Species'],
                       columns=['Irrigation'],
                       aggfunc=['count', 'sum'],
                       margins=True,
                       margins_name="Total")

file_name = PROJECT_PATH + "Report/v02/PivotTable_Region_Species_Irrigation_Count_Area.xlsx"
tmp.to_excel(file_name)


#  Report: Irrigation - Species - Region - District
tmp = data.pivot_table(values="Irrigation_Area",
                       index=['Region', 'District', 'Species'],
                       columns=['Irrigation'],
                       aggfunc=['count', 'sum'],
                       margins=True,
                       margins_name="Total")

file_name = PROJECT_PATH + \
    "Report/v02/PivotTable_Region_District_Species_Irrigation_Count_Area.xlsx"
tmp.to_excel(file_name)


#  Check Peyman Area
tmpG = data.groupby(['Region', 'District', 'Peyman'])

tmp = tmpG.agg({
    'Peyman_Area': ['min', 'max']
}).reset_index()

tmp.columns = ['Region', 'District', 'Peyman',
               'Peyman_Area_min', 'Peyman_Area_max']

tmp = tmp >> define(Check=if_else('Peyman_Area_min == Peyman_Area_max',
                                  True,
                                  False))

tmp.to_excel(PROJECT_PATH + "Report/v02/Peyman_Area_Check.xlsx", index=False)

   منطقه ناحیه پیمان                   نام پیمان  تعداد ردیف
0     01    01    01         الف ناحیه 1 منطقه 1        1325
1     01    02    01  الف ناحیه 2 منطقه 1 - 9808        1294
2     01    02    02    ب ناحیه 2 منطقه 1 - 9808        1126
3     01    03    01      ناحیه 3 منطقه 1 - 9711        1110
4     02    01    01  الف ناحیه 1 منطقه 2 - 9809         632
..   ...   ...   ...                         ...         ...
72    15    01    01        ناحیه 1 کمربند شمالی           3
73    16  10.2    01   ناحیه 10.2 سازمان پارک‌ها           8
74    16  11.2    01   ناحیه 11.2 سازمان پارک‌ها           8
75    16  12.2    01   ناحیه 12.2 سازمان پارک‌ها           4
76    16   7.1    01    ناحیه 7.1 سازمان پارک‌ها          28

[77 rows x 5 columns]
     منطقه ناحیه پیمان  لکه  \
0       01    01    01  001   
1       01    01    01  002   
2       01    01    01  003   
3       01    01    01  004   
4       01    01    01  005   
...    ...   ...   ...  ...   
2295    16  10.2    01  001 

In [ ]:
#  ETL Calculation - Efficacy Class
Eff = {
    '01': 0.70,
    '02': 0.60,
    '03': 0.80,
    '04': 0.80
}

data['Eff'] = list(map(Eff.get, data['Irrigation']))


#  ETL Calculation - Microclimate Class
Kmc = {
    '01': 1.20,
    '02': 1.20,
    '03': 1.15,
    '04': 1.10,
    '05': 1.10,
    '06': 1.05,
    '07': 1.05,
    '08': 1.00,
    '09': 0.90,
    '10': 0.80,
    '11': 1.00,
    '12': 1.05,
    '13': 1.05
}

data['Kmc'] = list(map(Kmc.get, data['Spot']))


#  ETL Calculation - Species Plant Class
Ksp = {
    '01': 0.70,
    '02': 0.75,
    '04': 0.80,
    '05': 0.80,
    '06': 0.80,
    '10': 0.70,
    '12': 0.49,
    '13': 0.00,
    '14': 0.80
}

data['Ksp'] = list(map(Ksp.get, data['Species']))


#  ETL Calculation - Change Species Plant Class In Year (1)
KspChangeYear = {
    '01': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '02': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '04': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '05': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '06': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '10': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '12': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40],
    '13': [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    '14': [0.60, 0.75, 0.90, 1.00, 0.95, 0.80, 0.60, 0.45, 0.35, 0.25, 0.30, 0.40]
}

Ksp_Month_Name = ['Ksp_M01', 'Ksp_M02', 'Ksp_M03', 'Ksp_M04', 'Ksp_M05', 'Ksp_M06',
                  'Ksp_M07', 'Ksp_M08', 'Ksp_M09', 'Ksp_M10', 'Ksp_M11', 'Ksp_M12']

KspChangeYear = pd.DataFrame(KspChangeYear, index=Ksp_Month_Name).T

KspChangeYear = KspChangeYear.reset_index()
KspChangeYear = KspChangeYear.rename(columns={'index': 'Species'})

data = data >> left_join(KspChangeYear, on='Species')


#  ETL Calculation - Change Species Plant Class In Year (2)
tmpG = data.groupby(by=['Region', 'District', 'Peyman',
                        'Address', 'Irrigation'])['Irrigation_Area']

tmp = tmpG.sum().reset_index().rename(
    columns={'Irrigation_Area': 'SubIrrigation_Area'}
)

data = data >> left_join(tmp,
                         on=['Region', 'District', 'Peyman', 'Address', 'Irrigation'])

data = data >> define(
    Ksp_M01='Ksp_M01 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M02='Ksp_M02 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M03='Ksp_M03 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M04='Ksp_M04 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M05='Ksp_M05 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M06='Ksp_M06 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M07='Ksp_M07 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M08='Ksp_M08 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M09='Ksp_M09 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M10='Ksp_M10 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M11='Ksp_M11 * Ksp * Irrigation_Area / SubIrrigation_Area')
data = data >> define(
    Ksp_M12='Ksp_M12 * Ksp * Irrigation_Area / SubIrrigation_Area')

tmpG = data.groupby(
    by=['Region', 'District', 'Peyman', 'Address', 'Irrigation'])

tmpG = tmpG['Ksp_M01', 'Ksp_M02', 'Ksp_M03', 'Ksp_M04', 'Ksp_M05', 'Ksp_M06',
            'Ksp_M07', 'Ksp_M08', 'Ksp_M09', 'Ksp_M10', 'Ksp_M11', 'Ksp_M12']

tmp = tmpG.sum().reset_index()

tmp = tmp.rename(columns={
    'Ksp_M01': 'Ksp_M01_Sum_SubIrigation',
    'Ksp_M02': 'Ksp_M02_Sum_SubIrigation',
    'Ksp_M03': 'Ksp_M03_Sum_SubIrigation',
    'Ksp_M04': 'Ksp_M04_Sum_SubIrigation',
    'Ksp_M05': 'Ksp_M05_Sum_SubIrigation',
    'Ksp_M06': 'Ksp_M06_Sum_SubIrigation',
    'Ksp_M07': 'Ksp_M07_Sum_SubIrigation',
    'Ksp_M08': 'Ksp_M08_Sum_SubIrigation',
    'Ksp_M09': 'Ksp_M09_Sum_SubIrigation',
    'Ksp_M10': 'Ksp_M10_Sum_SubIrigation',
    'Ksp_M11': 'Ksp_M11_Sum_SubIrigation',
    'Ksp_M12': 'Ksp_M12_Sum_SubIrigation'
})

data = data >> left_join(tmp,
                         on=['Region', 'District', 'Peyman', 'Address', 'Irrigation'])


#  ETL Calculation - Density Plant Class
Kd = {
    1: 1,
    2: 1,
    3: 1.2,
    4: 1.2,
    5: 1.3,
    6: 1.3,
    7: 1.3,
    8: 1.3
}


def name(x):
    x = list(x)
    if "10" in x and "13" in x:
        return -1
    else:
        return 0


tmpAG = data.groupby(
    by=['Region', 'District', 'Peyman', 'Address', 'Irrigation'])
tmpA = tmpAG.size().reset_index().rename(columns={0: 'Species_Count'})

tmpBG = data.groupby(by=['Region', 'District', 'Peyman', 'Address', 'Extera'])
tmpB = tmpBG.size().reset_index().rename(columns={0: 'Asleh'})

tmpC = tmpA >> left_join(tmpB, on=['Region', 'District', 'Peyman', 'Address'])
tmpC = tmpC >> call(pd.DataFrame.fillna, 0)

tmpDG = data.groupby(by=['Region', 'District', 'Peyman', 'Address'])
tmpD = tmpDG.agg({'Species': name})
tmpD = tmpD.reset_index().rename(columns={'Species': 'Double_Asleh'})

tmpE = tmpC >> left_join(tmpD, on=['Region', 'District', 'Peyman', 'Address'])
tmpE = tmpE >> define(Species_Count='Species_Count + Asleh + Double_Asleh')
tmpE = tmpE >> select('Extera', 'Asleh', 'Double_Asleh', drop=True)

data = data >> left_join(tmpE,
                         on=['Region', 'District', 'Peyman', 'Address', 'Irrigation'])

data['Kd'] = list(map(Kd.get, data['Species_Count']))


#  ETL Calculation - KL Calculate
data = data >> define(KL_M01='Ksp_M01_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M02='Ksp_M02_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M03='Ksp_M03_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M04='Ksp_M04_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M05='Ksp_M05_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M06='Ksp_M06_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M07='Ksp_M07_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M08='Ksp_M08_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M09='Ksp_M09_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M10='Ksp_M10_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M11='Ksp_M11_Sum_SubIrigation * Kmc * Kd')
data = data >> define(KL_M12='Ksp_M12_Sum_SubIrigation * Kmc * Kd')

In [ ]:
#  ETL Calculation

del tmpG, tmp

ETP = {
    'Dry': [122.6, 176.6, 220.8, 255.8, 236.9, 185.8, 123.7, 64.2, 41.9, 35.3, 42.0, 79.9],
    'Normal': [116.1, 168.9, 206.6, 231.3, 201.4, 172, 104.2, 56.9, 38, 39.8, 49.6, 81.1],
    'Wet': [103.4, 151.3, 196.2, 219.3, 199.0, 157.9, 109.0, 63.8, 38.7, 27.8, 32.7, 60.7]
}

Prec = {
    'Dry': [19.6, 14.6, 5.8, 0.5, 0.0, 0.1, 2.6, 7.4, 11.4, 21.1, 21.3, 23.2],
    'Normal': [22.9, 16.3, 5.5, 0.1, 0.0, 0.0, 6.0, 4.3, 3.5, 3.4, 2.2, 30.1],
    'Wet': [30.4, 19.5, 5.3, 0.5, 0.1, 0.3, 0.6, 3.6, 14.1, 23.5, 25.7, 28.6]
}

for i in ['Dry', 'Normal', 'Wet']:

    ETL_Info = {
        'Number_Day': [31, 31, 31, 31, 31, 31, 30, 30, 30, 30, 30, 29],
        'PET': ETP[i],
        'Eff_Precipitation': Prec[i],
        'Ground_Water': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        'Deficit_Irrigation': [0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95],
        'LF': [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
        'Shading_Area': [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
        'Percentage_Wetted_Area': [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
    }

    Month_Name = ['M01', 'M02', 'M03', 'M04', 'M05', 'M06',
                  'M07', 'M08', 'M09', 'M10', 'M11', 'M12']

    ETL_Info = pd.DataFrame(ETL_Info, index=Month_Name)

    tmpG = data.groupby(
        by=['Region', 'District', 'Peyman', 'Address', 'Irrigation']
    )

    tmpG = tmpG['KL_M01', 'KL_M02', 'KL_M03', 'KL_M04', 'KL_M05', 'KL_M06',
                'KL_M07', 'KL_M08', 'KL_M09', 'KL_M10', 'KL_M11', 'KL_M12',
                'SubIrrigation_Area', 'Eff']

    tmp = tmpG.mean().reset_index()

    month = "M01"
    tmp = tmp >> define(ETL_M01='(KL_M01 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M01=if_else('ETL_M01 < 0', 0, 'ETL_M01'))
    tmp = tmp >> define(
        IrriReq_M01='ETL_M01 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M02"
    tmp = tmp >> define(ETL_M02='(KL_M02 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M02=if_else('ETL_M02 < 0', 0, 'ETL_M02'))
    tmp = tmp >> define(
        IrriReq_M02='ETL_M02 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M03"
    tmp = tmp >> define(ETL_M03='(KL_M03 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M03=if_else('ETL_M03 < 0', 0, 'ETL_M03'))
    tmp = tmp >> define(
        IrriReq_M03='ETL_M03 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M04"
    tmp = tmp >> define(ETL_M04='(KL_M04 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M04=if_else('ETL_M04 < 0', 0, 'ETL_M04'))
    tmp = tmp >> define(
        IrriReq_M04='ETL_M04 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M05"
    tmp = tmp >> define(ETL_M05='(KL_M05 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M05=if_else('ETL_M05 < 0', 0, 'ETL_M05'))
    tmp = tmp >> define(
        IrriReq_M05='ETL_M05 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M06"
    tmp = tmp >> define(ETL_M06='(KL_M06 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M06=if_else('ETL_M06 < 0', 0, 'ETL_M06'))
    tmp = tmp >> define(
        IrriReq_M06='ETL_M06 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M07"
    tmp = tmp >> define(ETL_M07='(KL_M07 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M07=if_else('ETL_M07 < 0', 0, 'ETL_M07'))
    tmp = tmp >> define(
        IrriReq_M07='ETL_M07 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M08"
    tmp = tmp >> define(ETL_M08='(KL_M08 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M08=if_else('ETL_M08 < 0', 0, 'ETL_M08'))
    tmp = tmp >> define(
        IrriReq_M08='ETL_M08 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M09"
    tmp = tmp >> define(ETL_M09='(KL_M09 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M09=if_else('ETL_M09 < 0', 0, 'ETL_M09'))
    tmp = tmp >> define(
        IrriReq_M09='ETL_M09 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M10"
    tmp = tmp >> define(ETL_M10='(KL_M10 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M10=if_else('ETL_M10 < 0', 0, 'ETL_M10'))
    tmp = tmp >> define(
        IrriReq_M10='ETL_M10 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M11"
    tmp = tmp >> define(ETL_M11='(KL_M11 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M11=if_else('ETL_M11 < 0', 0, 'ETL_M11'))
    tmp = tmp >> define(
        IrriReq_M11='ETL_M11 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    month = "M12"
    tmp = tmp >> define(ETL_M12='(KL_M12 * ETL_Info.loc[month]["PET"] * ETL_Info.loc[month]["Deficit_Irrigation"] * ETL_Info.loc[month]["Shading_Area"] - (ETL_Info.loc[month]["Eff_Precipitation"] + ETL_Info.loc[month]["Ground_Water"]) * ETL_Info.loc[month]["Percentage_Wetted_Area"] / 100) / ETL_Info.loc[month]["Number_Day"] / (1 - ETL_Info.loc[month]["LF"]) / Eff')
    tmp = tmp >> define(ETL_M12=if_else('ETL_M12 < 0', 0, 'ETL_M12'))
    tmp = tmp >> define(
        IrriReq_M12='ETL_M12 * ETL_Info.loc[month]["Number_Day"] * SubIrrigation_Area / 1000 / 1000'
    )

    tmp = tmp >> define(ETL_YEAR='ETL_M01 * ETL_Info.loc["M01"]["Number_Day"] + ETL_M02 * ETL_Info.loc["M02"]["Number_Day"] + ETL_M03 * ETL_Info.loc["M03"]["Number_Day"] + ETL_M04 * ETL_Info.loc["M04"]["Number_Day"] + ETL_M05 * ETL_Info.loc["M05"]["Number_Day"] + ETL_M06 * ETL_Info.loc["M06"]["Number_Day"] + ETL_M07 * ETL_Info.loc["M07"]["Number_Day"] + ETL_M08 * ETL_Info.loc["M08"]["Number_Day"] + ETL_M09 * ETL_Info.loc["M09"]["Number_Day"] + ETL_M10 * ETL_Info.loc["M10"]["Number_Day"] + ETL_M11 * ETL_Info.loc["M11"]["Number_Day"] + ETL_M12 * ETL_Info.loc["M12"]["Number_Day"]')
    tmp = tmp >> define(
        IrriReq_YEAR='ETL_YEAR * SubIrrigation_Area / 1000 / 1000'
    )

    tmp = tmp >> select(
        'Region', 'District', 'Peyman', 'Address', 'Irrigation', 'SubIrrigation_Area',
        'ETL_M01', 'ETL_M02', 'ETL_M03',
        'ETL_M04', 'ETL_M05', 'ETL_M06',
        'ETL_M07', 'ETL_M08', 'ETL_M09',
        'ETL_M10', 'ETL_M11', 'ETL_M12',
        'IrriReq_M01', 'IrriReq_M02', 'IrriReq_M03',
        'IrriReq_M04', 'IrriReq_M05', 'IrriReq_M06',
        'IrriReq_M07', 'IrriReq_M08', 'IrriReq_M09',
        'IrriReq_M10', 'IrriReq_M11', 'IrriReq_M12',
        'ETL_YEAR', 'IrriReq_YEAR'
    )

    tmp.columns = ['Region', 'District', 'Peyman', 'Address',
                   'Irrigation', 'SubIrrigation_Area'] + list(i + '_' + tmp.columns[6:])

    try:
        final_result
    except NameError:
        final_result = tmp
    else:
        tmp = tmp >> select('-SubIrrigation_Area')
        final_result = pd.merge(final_result,
                                tmp,
                                how='left',
                                on=['Region', 'District', 'Peyman', 'Address', 'Irrigation'])

final_result.to_excel(PROJECT_PATH + "Report/v02/Final_Result.xlsx",
                      index=False)

In [ ]:
final_result

In [ ]:
#  Load Shapefiles
region_shp = PROJECT_PATH + 'Data/Mashhad_City_Layers/Region/ShapeFile/Regions.shp'
district_shp = PROJECT_PATH + \
    'Data/Mashhad_City_Layers/District/ShapeFile/Districts.shp'

region_df = gpd.read_file(filename=region_shp)[['Name', 'geometry']]
district_df = gpd.read_file(filename=district_shp)[['Name', 'geometry']]

region_df.columns = ['ID', 'geometry']
district_df.columns = ['ID', 'geometry']

In [ ]:
# raw_data = raw_data >> call('.dropna', subset=['ID'])